In [10]:
import numpy as np
import toughio
import sys

# --- CONFIGURATION ---
MESH_FILE = "MESH"
OUTPUT_FILE = "INCON"

# Gradient Settings
P_SURFACE = 1.013e5   # Pa (1 atm)
T_SURFACE = 15.0      # C
GRAD_T    = 0.03      # C/m
RHO_W     = 1000.0    # kg/m3
SALINITY  = 0.1       # Mass Fraction

print(f"--- Generating Standalone {OUTPUT_FILE} ---")

# 1. Read Mesh
try:
    mesh_data = toughio.read_mesh(MESH_FILE)
    if hasattr(mesh_data, 'centers'):
        labels, centers = mesh_data.labels, mesh_data.centers
    else:
        # Fallback for dictionary format
        eleme = mesh_data.get('elements', mesh_data.get('ELEME'))
        labels = np.array(list(eleme.keys()))
        centers = np.array([v['center'] for v in eleme.values()])
except Exception as e:
    print(f"Error reading MESH: {e}")
    sys.exit(1)

# 2. Calculate Top of Model (Surface)
z_vals = centers[:, 2]
max_z = np.max(z_vals)
print(f"Detected Surface Elevation: {max_z:.1f} m")

# 3. Write INCON File
with open(OUTPUT_FILE, 'w', newline='\n') as f:
    f.write("INCON----1----*----2----*----3----*----4----*----5----*----6----*----7----*----8\n")
    
    for i, label in enumerate(labels):
        # Calculate Depth
        depth = max_z - z_vals[i]
        if depth < 0: depth = 0
        
        # Calculate Physics
        p = P_SURFACE + (RHO_W * 9.81 * depth)
        t = T_SURFACE + (GRAD_T * depth)
        
        # WRITE ENTRY
        # Line 1: Block Label (Left aligned in 5 chars)
        f.write(f"{label:<5}\n")
        
        # Line 2: Variables (Strict 20-char width for formatting safety)
        # Format: Pressure, Salinity, GasSat(0), Temperature
        # standard TOUGH format is often Free Format, but Fixed (4E20.14) is safest.
        f.write(f" {p:.4E} {SALINITY:.4E} {0.0:.4E} {t:.4E}\n")

print(f"Done! '{OUTPUT_FILE}' created with {len(labels)} entries.")


--- Generating Standalone INCON ---
Detected Surface Elevation: 1400.0 m
Done! 'INCON' created with 18328 entries.
